In [2]:
import numpy as np
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
actions = [
    'flower',
    'crown',
    'heart_beat',
    'firework',
    'bear',
    'cat',
    'son_celebration',
    'Hear_on_the_cheek',
    'gun',
    'pipe',
    'tiger',
    'landmarks'
]

data = np.concatenate([
    np.load('new_dataset/seq_flower_1750130471.npy'),
    np.load('new_dataset/seq_crown_1750130505.npy'),
    np.load('new_dataset/seq_heart_beat_1750139892.npy'),
    np.load('new_dataset/seq_firework_1750140689.npy'),
    np.load('new_dataset/seq_bear_1750140330.npy'),
    np.load('new_dataset/seq_cat_1750140596.npy'),
    np.load('new_dataset/seq_son_celebration_1750139350.npy'),
    np.load('new_dataset/seq_Hear_on_the_cheek_1750139384.npy'),
    np.load('new_dataset/seq_gun_1750139891.npy'),
    np.load('new_dataset/seq_pipe_1750140043.npy'),
    np.load('new_dataset/seq_tiger_1750140076.npy'),
    np.load('new_dataset/seq_landmarks_1750140109.npy'),
], axis=0)

data.shape

(4401, 30, 199)

In [4]:
x_data = data[:, :, :-1]
labels = data[:, 0, -1]

print(x_data.shape)
print(labels.shape)

(4401, 30, 198)
(4401,)


In [5]:
from tensorflow.keras.utils import to_categorical

# float로 되어 있다면 반드시 int로 변환
labels = np.array(labels).astype(int)

# num_classes 정확히 계산
num_classes = int(np.max(labels)) + 1

# 원-핫 인코딩
y_data = to_categorical(labels, num_classes=num_classes)
print(y_data.shape)

(4401, 12)


In [6]:
from sklearn.model_selection import train_test_split

x_data = x_data.astype(np.float32)
y_data = y_data.astype(np.float32)

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=2021)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(3960, 30, 198) (3960, 12)
(441, 30, 198) (441, 12)


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

model = Sequential([
    LSTM(128, input_shape=x_train.shape[1:3]),
    Dense(32, activation='relu'),
    Dense(len(actions), activation='softmax')
])

optimizer = Adam(learning_rate=0.05)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               167424    
_________________________________________________________________
dense (Dense)                (None, 32)                4128      
_________________________________________________________________
dense_1 (Dense)              (None, 12)                396       
Total params: 171,948
Trainable params: 171,948
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

EPOCHS = 30

history = model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=EPOCHS,
    callbacks=[
        ModelCheckpoint('models/model.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=50, verbose=1, mode='auto')
    ]
)

Epoch 1/30
124/124 [==============================] - 4s 22ms/step - loss: 1.9262 - acc: 0.3154 - val_loss: 1.1003 - val_acc: 0.5238

Epoch 00001: val_acc improved from -inf to 0.52381, saving model to models\model.h5
Epoch 2/30
124/124 [==============================] - 2s 14ms/step - loss: 1.2218 - acc: 0.5205 - val_loss: 1.3385 - val_acc: 0.4331

Epoch 00002: val_acc did not improve from 0.52381
Epoch 3/30
124/124 [==============================] - 2s 14ms/step - loss: 1.1956 - acc: 0.4985 - val_loss: 1.0910 - val_acc: 0.6327

Epoch 00003: val_acc improved from 0.52381 to 0.63265, saving model to models\model.h5
Epoch 4/30
124/124 [==============================] - 2s 15ms/step - loss: 1.1586 - acc: 0.5717 - val_loss: 1.0009 - val_acc: 0.6644

Epoch 00004: val_acc improved from 0.63265 to 0.66440, saving model to models\model.h5
Epoch 5/30
124/124 [==============================] - 2s 14ms/step - loss: 1.1336 - acc: 0.5910 - val_loss: 0.9412 - val_acc: 0.6281

Epoch 00005: val_acc d

In [ ]:
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots(figsize=(16, 10))
acc_ax = loss_ax.twinx()

loss_ax.plot(history.history['loss'], 'y', label='train loss')
loss_ax.plot(history.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(history.history['acc'], 'b', label='train acc')
acc_ax.plot(history.history['val_acc'], 'g', label='val acc')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='upper left')

plt.show()

In [8]:
from sklearn.metrics import multilabel_confusion_matrix
from tensorflow.keras.models import load_model

model = load_model('models/model.h5')

y_pred = model.predict(x_val)

multilabel_confusion_matrix(np.argmax(y_val, axis=1), np.argmax(y_pred, axis=1))

array([[[378,  14],
        [ 26,  23]],

       [[402,   1],
        [  5,  33]],

       [[388,  10],
        [  3,  40]],

       [[412,   0],
        [ 29,   0]],

       [[408,   1],
        [ 32,   0]],

       [[408,   1],
        [ 13,  19]],

       [[333,  63],
        [  0,  45]],

       [[386,  13],
        [  1,  41]],

       [[413,   1],
        [  4,  23]],

       [[386,  19],
        [  2,  34]],

       [[379,  25],
        [  2,  35]],

       [[410,   0],
        [ 31,   0]]], dtype=int64)